In [1]:
import os

In [2]:
pwd

'/home/lucky/Documents/data_science_interview/Udemy/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/home/lucky/Documents/data_science_interview/Udemy/Chicken-Disease-Classification'

# Entity

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [28]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

# Config Manager

In [29]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [30]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

# Component callback and training

In [31]:
import time

In [32]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [33]:
import os
import tensorflow as tf
import time

In [34]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [35]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-12-08 10:29:46,814: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-08 10:29:46,817: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-08 10:29:46,818: INFO: common: created directory at: artifacts]
[2023-12-08 10:29:46,820: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-12-08 10:29:46,821: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-12-08 10:29:46,823: INFO: common: created directory at: artifacts/training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/20


2023-12-08 10:29:48.416358: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-12-08 10:29:48.526189: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-12-08 10:29:48.899889: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-12-08 10:29:49.056926: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


 1/19 [>.............................] - ETA: 53s - loss: 0.6663 - accuracy: 0.6250

2023-12-08 10:29:50.903108: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


19/19 [==============================] - 65s 3s/step - loss: 14.4576 - accuracy: 0.5169 - val_loss: 3.1620 - val_accuracy: 0.6094
Epoch 2/20


/home/lucky/Documents/data_science_interview/Udemy/Chicken-Disease-Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 65s 3s/step - loss: 10.1384 - accuracy: 0.5507 - val_loss: 20.0901 - val_accuracy: 0.6094
Epoch 3/20
19/19 [==============================] - 63s 3s/step - loss: 5.3647 - accuracy: 0.7061 - val_loss: 24.7810 - val_accuracy: 0.3906
Epoch 4/20
19/19 [==============================] - 66s 3s/step - loss: 1.5218 - accuracy: 0.8953 - val_loss: 1.2569 - val_accuracy: 0.8750
Epoch 5/20
19/19 [==============================] - 64s 3s/step - loss: 0.7500 - accuracy: 0.9054 - val_loss: 0.8378 - val_accuracy: 0.9219
Epoch 6/20
19/19 [==============================] - 64s 3s/step - loss: 0.8081 - accuracy: 0.8953 - val_loss: 2.2767 - val_accuracy: 0.7344
Epoch 7/20
19/19 [==============================] - 63s 3s/step - loss: 2.2649 - accuracy: 0.7905 - val_loss: 1.6032 - val_accuracy: 0.8281
Epoch 8/20
19/19 [==============================] - 63s 3s/step - loss: 0.7002 - accuracy: 0.8953 - val_loss: 0.7218 - val_accuracy: 0.9219
Epoch 9/20
19/19 [==========

KeyboardInterrupt: 